In [2]:
from openpyxl import Workbook
import pandas as pd
import re
import xlwings

In [3]:
columna = ['cod_alumno','ape_paterno','ape_materno','nom_alumno','tel_alumno','movil_alumno','coe_alumno','coe_alu_personal','tipo_doc','doc_alumno','año_ingreso','periodo_egreso','escuela']
estudiantes = ['cod_alumno','tipo_doc','doc_alumno','nom_alumno','ape_paterno','ape_materno','escuela']
temporal = ['coe_alumno','coe_alumno_personal','tel_alumno','movil_alumno','doc_ingreso','año_ingreso','periodo_egreso']
likert = ['valor','descripcion']

In [4]:
conta = pd.read_excel("./EGRESADOS/BASE DE DATOS CONTABILIDAD EGRESADOS.xlsx",sheet_name="Hoja1"    )
gestion = pd.read_excel("./EGRESADOS/BASE DE DATOS EGRESADOS GESTIÓN PÚBLICA.xlsx",sheet_name="Hoja1")
audit = pd.read_excel("./EGRESADOS/BASE DE DATOS AUDITORIA EGRESADOS.xlsx",sheet_name="Hoja1")

In [5]:
conta['escuela']='contabilidad'
gestion['escuela']='gestión pública'
audit['escuela']='auditoría'

In [6]:
db = pd.concat([conta,gestion,audit]).reset_index(drop=True)

In [7]:
for col in db.columns:
    if col == "anio_ingreso":
        db.rename(columns={'anio_ingreso':'año_ingreso'},inplace=True)
    if col == 'did_alumno':
        db.rename(columns={'did_alumno':'doc_alumno'},inplace=True)
    if col == 'tel_alu_movil':
        db.rename(columns={'tel_alu_movil':'movil_alumno'},inplace=True)
    if col == 'des_doc_identidad':
        db.rename(columns={'des_doc_identidad':'tipo_doc'},inplace=True)

In [8]:
for col in db.columns:
    if col not in columna:
        db.drop(col,inplace=True,axis=1)

In [9]:
db = db.drop_duplicates().reset_index(drop=True)

In [10]:
for col in db.columns:
    db[col]=db[col].astype('string',copy=True,errors='raise')

In [11]:
db.periodo_egreso.replace('----',None,inplace=True)
db.periodo_egreso.replace('00000',None,inplace=True)
db.tel_alumno.replace('-',None,inplace=True)
db.tel_alumno.replace('0',None,inplace=True)
db.año_ingreso.replace('0',None,inplace=True)


In [12]:
db.fillna('sin_registro',inplace=True)

In [13]:
def reemplazo_tel(val):
    if val[3]=='-':
        val=val[:3]+val[4:]
    if re.search('01',val):
        val=val[2:]
    if len(val)!=7 or val[0]=='0':
        val='sin_registro'
    if not re.search('^\d+',val) and val!='sin_registro':
        val='sin_registro'
    return val
def reemplazo_mov(val):
    if val[5]=='-':
        val=val[:5]+val[6:]
    if len(val)!=9 or val[0]!= '9':
        val='sin_registro'
    return val
def reemplazo_año_ingreso(val):
    if len(val)!=4:
        val='sin_registro'
    return val
def reemplazo_periodo_egreso(val):
    if val!='sin_registro' and val[4]!='-':
        val=val[:4]+'-'+val[-1]
    return val
def correct_doc_alumno(val):
    if len(val)!=8 and len(val)!=10:
        val="sin_registro"
    if len(val)==10 and val[8]=='.':
        val=val[:8]
    return val

In [15]:
db.tel_alumno = db.tel_alumno.apply(reemplazo_tel)
db.movil_alumno = db.movil_alumno.apply(reemplazo_mov)
db.año_ingreso = db.año_ingreso.apply(reemplazo_año_ingreso)
db.periodo_egreso = db.periodo_egreso.apply(reemplazo_periodo_egreso)
db.doc_alumno = db.doc_alumno.apply(correct_doc_alumno)

In [101]:
db2 = db[db.doc_alumno!="sin_registro"]

In [105]:
egresados = db2[columna]
egresados.columns = egresados.columns.str.upper()
egresados = egresados.style.set_table_styles([{'selector': 'th', 'props': [('text-align', 'center')]}]).set_properties(**{'text-align': 'center'})

In [106]:
egresados.to_excel('EGRESADOS.xlsx', sheet_name = 'EGRESADOS', index=False)
with xlwings.App(visible=True) as app:
    wb = xlwings.Book('EGRESADOS.xlsx')
    for sheet in range(len(wb.sheets)):
        ws = wb.sheets[sheet]
        ws.autofit()
    wb.save()
    wb.close()